In [1]:
using Distributions
using Random
using LinearAlgebra
using Plots
using StatsBase
using LaTeXStrings
using DataFrames
using CSV
using Base.Threads
using HDF5
gr()

Plots.GRBackend()

In [2]:
# サンプリングパラメータ読み込み
function read_para(read_path)
    para_df = CSV.read(read_path, DataFrame)
    para = Matrix(para_df[:,:])
    return para
end

# 本データ読み込み
function read_file_spc(file_path)
    file = h5open(file_path, "r")
    data = read(file, "spc")
    return data
end

read_file_spc (generic function with 1 method)

In [3]:
function make_beta(gamma,L)
    #最初は等差で決定
    beta_list = gamma.^( (2:1:L) .- (L) )
    beta_list = pushfirst!(beta_list, 0.0)
    return beta_list
end

make_beta (generic function with 1 method)

In [37]:
#データ数
function data_num_spc()
    n_spc = 350
    return n_spc
end
n_spc = data_num_spc()

#各種結晶場パラメータ
function ini()
    # Hund's Rule Ground J-Multiplet Ce3+ n4f=1
    n4f = 1.0
    L = 3.0
    S = 0.5
    J = L - S
    g = 1.0 + (J * (J + 1.0) + S * (S + 1.0) - L * (L + 1.0)) / (2.0 * J * (J + 1.0))
    Jz = [2.5, 1.5, 0.5, -0.5, -1.5, -2.5]
    return J, g, Jz
end

#温度配列
function temp()
    Temp_table_spc = collect(0.2:0.2:70) # length 350
    return Temp_table_spc
end

#結晶場行列
function Onn_make(B40)
    # O40
    O40_vec_x = [60.0, -180.0, 120.0, 120.0, -180.0, 60.0]
    O40_vec = O40_vec_x * B40
    O40 = diagm(0 => O40_vec)
    Onn = O40
    # O44
    B44 = 5 * B40
    O44_value = sqrt(120.0 * 24.0) * B44 / 2.0
    Onn[5,1] = O44_value
    Onn[6,2] = O44_value
    Onn[1,5] = O44_value
    Onn[2,6] = O44_value
    return Onn
end

function spc(Onn)
    #パラメータ読み込み
    _, g, _ = ini()

    # 非対角要素
    Hmag_vec_1 = sqrt.([5,8,9,8,5]) * (1 + 1*im) * 5 * g * 0.67171 / 2
    Hmag_1 = diagm(1 => Hmag_vec_1)
    Hmag_2 = diagm(-1 => conj.(Hmag_vec_1))
    Hmag = Hmag_1 + Hmag_2

    #結晶場＋磁場ハミルトニアンの行列要素
    H = Onn + Hmag

    eigval, eigvec = eigen(H)

    Temp_table_spc = temp()
    SpcHeat_Temp = zeros(length(Temp_table_spc))
    
    @inbounds for (i, Temp) in enumerate(Temp_table_spc)
        eigval_2 = - eigval / Temp
        eigval_2_max = maximum(eigval_2)
        eigval_ratio = eigval_2 .- eigval_2_max
        exp_eigval = exp.(eigval_ratio)

        Z0 = sum(exp_eigval)
        Z1 = sum(eigval_2 .* exp_eigval)
        Z2 = sum(eigval_2.^2 .* exp_eigval)
        
        SpcHeat=(- (Z1/Z0)^2 + (Z2/Z0) )*8.31441
        SpcHeat_Temp[i] = SpcHeat
    end
    return SpcHeat_Temp
end

function error_spc(B40, SpcHeat_Temp_noise)
    error_value = sum((SpcHeat_Temp_noise - spc(Onn_make(B40))).^2)/(2*n_spc)
    return error_value
end

function error_spc_list(para_list, SpcHeat_Temp_noise)
    error_list = zeros(size(para_list)[1],size(para_list)[2])
    @threads for i in 1:size(para_list)[1]
        for j in 1:size(para_list)[2]
            error_list[i,j] = error_spc(para_list[i,j], SpcHeat_Temp_noise)
        end
    end
    return error_list
end

error_spc_list (generic function with 1 method)

In [38]:
function find_free_energy_spc(error, b_spc, gamma, L)
    # 温度リスト
    beta = make_beta(gamma, L)
    #温度差
    beta_dif = beta[2:end] - beta[1:end-1]
    
    #温度差をかける
    for replica in 1:L-1
        error[:,replica] *= beta_dif[replica]
    end
    
    #n,noiseをかける
    error *= - n_spc * b_spc
    
    #最大値取得
    error_max = zeros(L-1)
    for replica in 1:L-1
        error_max[replica] = maximum(error[:,replica])
    end
    
    #最大値で引いたエネルギー関数
    error_dif = copy(error)
    for replica in 1:L-1
        error_dif[:,replica] = error[:,replica] .- error_max[replica]
    end
    
    #期待値計算
    log_exp_sum = zeros(L-1)
    for replica in 1:L-1
        log_exp_sum[replica] = log(sum(exp.(error_dif)[:,replica]))
    end
    
    #各レプリカでの自由エネルギー期待値
    free_energy_for_replica = -(- log(size(error)[1]) .+  error_max .+ log_exp_sum)
    
    #最終的な自由エネルギー(足し算)
    free_energy = sum(free_energy_for_replica) - n_spc/2*(log(b_spc)-log(2*pi))
    
    return free_energy
end

find_free_energy_spc (generic function with 1 method)

In [49]:
function main(para_path, data_path, save_path)
    para = read_para(para_path)
    SpcHeat_Temp_noise = read_file_spc(data_path)

    b_spc = 10^(5)
    error_list = error_spc_list(para, SpcHeat_Temp_noise)

    #レプリカ数
    L = 40
    #逆温度間隔決定
    gamma = 1.4
    # 自由エネルギー
    free_energy = find_free_energy_spc(error_list, b_spc, gamma, L)

    # データフレームに書き出し
    df_error = DataFrame(error_list, :auto)
    df_error |> CSV.write(save_path * "error.csv",writeheader=true)

    free_path = save_path * "free.h5"
    h5open(free_path, "w") do file
        write(file,"free",free_energy)
    end

    print("free_energy=", free_energy)
end

main (generic function with 3 methods)

In [50]:
data_path = "/Users/test/home/lab_research_1/data_make/data/spc/spc_1.h5"

para_path = "/Users/test/home/lab_research_1/exmc/result/spc/spc1_B40.csv"

save_path = "/Users/test/home/lab_research_1/exmc/free_energy_check/spc/spc1_"

para1, error_list1, free_energy1 = main(para_path, data_path, save_path)

free_energy=136403.31305216477

MethodError: MethodError: no method matching iterate(::Nothing)
Closest candidates are:
  iterate(!Matched::Union{LinRange, StepRangeLen}) at range.jl:664
  iterate(!Matched::Union{LinRange, StepRangeLen}, !Matched::Int64) at range.jl:664
  iterate(!Matched::T) where T<:Union{Base.KeySet{var"#s79", var"#s78"} where {var"#s79", var"#s78"<:Dict}, Base.ValueIterator{var"#s77"} where var"#s77"<:Dict} at dict.jl:693
  ...

In [42]:
free_energy1

136403.31305216477

In [43]:
error_list1

100×40 Matrix{Float64}:
 4.7079e-6   1.89361e-6  2.69469e-6  …  1.01003e-6  3.16697e-6  4.94913e-6
 4.71739e-6  1.948e-6    2.89333e-6     2.26212e-6  3.16697e-6  2.38685e-5
 5.12051e-6  2.07077e-6  2.63576e-6     6.66041e-6  6.81956e-6  6.26109e-6
 5.08368e-6  1.88383e-6  2.71704e-6     1.88052e-6  3.16697e-6  6.26109e-6
 4.74382e-6  1.89595e-6  3.0271e-6      1.27777e-6  1.41404e-6  2.38685e-5
 4.80184e-6  2.21068e-6  2.63762e-6  …  1.01003e-6  9.32458e-6  2.38685e-5
 5.50013e-6  1.88329e-6  3.20325e-6     1.48136e-5  1.41404e-6  3.2636e-5
 5.42419e-6  2.70125e-6  2.66397e-6     9.97928e-6  1.78888e-6  1.10844e-5
 6.95008e-6  2.0237e-6   2.84686e-6     6.66041e-6  1.56017e-5  9.21456e-6
 4.96867e-6  2.02817e-6  4.17697e-6     1.11441e-5  2.10898e-6  4.94913e-6
 ⋮                                   ⋱                          
 6.58896e-6  1.93023e-6  4.99893e-6     5.60141e-5  3.9965e-5   3.2636e-5
 4.71739e-6  3.27173e-6  2.71304e-6     1.62647e-5  1.06201e-5  6.26109e-6
 7.83195e-6  

In [ ]:
## 採択率・交換率
df_error = DataFrame(error_list, :auto)
df_free_energy = DataFrame(free_energy, :auto)


df_ac_ex |> CSV.write(path * "error.csv",writeheader=true)
df_para_B40 |> CSV.write(path * "free_energy.csv",writeheader=true)